![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/02.0.Clinical_Assertion_Model.ipynb)

# Clinical Assertion Status Model


The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|


📌 The license was provided and all necessary libraries and jars were installed when the EMR cluster was created.


In [1]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd

# Automatically load jars to the session
spark = nlp.start()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1694591988531_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark Session already created, some configs may not take.
📋 Loading license number 0 from /lib/.johnsnowlabs/johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare.json

# Clinical Assertion Models (with pretrained models)

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | [assertion_dl](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_en.html)            |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  2 | [assertion_dl_biobert](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_biobert_en.html)    |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  3 | [assertion_dl_healthcare](https://nlp.johnsnowlabs.com/2020/09/23/assertion_dl_healthcare_en.html) |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  4 | [assertion_dl_large](https://nlp.johnsnowlabs.com/2020/05/21/assertion_dl_large_en.html)      |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  5 | [assertion_dl_radiology](https://nlp.johnsnowlabs.com/2021/03/18/assertion_dl_radiology_en.html)   |Confirmed, Suspected, Negative|
|  6 | [assertion_jsl](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_en.html)           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, Hypotetical|
|  7 | [assertion_jsl_large](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_large_en.html)     |present, absent, possible, planned, someoneelse, past, hypothetical|
|  8 |  [assertion_ml](https://nlp.johnsnowlabs.com/2020/01/30/assertion_ml_en.html) |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|
|  9 | [assertion_dl_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/17/assertion_dl_scope_L10R10_en_3_0.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 10 | [assertion_dl_biobert_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/24/assertion_dl_biobert_scope_L10R10_en_2_4.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 11 | [assertion_jsl_augmented](https://nlp.johnsnowlabs.com/2022/09/15/assertion_jsl_augmented_en.html)| Present, Absent, Possible, Planned, Past, Family, Hypotetical, SomeoneElse|




### Pretrained `assertion_jsl_augmented` model

In [2]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = medical.NerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["SYMPTOM","VS_FINDING","DISEASE_SYNDROME_DISORDER","ADMISSION_DISCHARGE","PROCEDURE"])

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = medical.AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)\
    .setEntityAssertion({
        "PROBLEM": ["hypothetical", "absent"],
        "treAtment": ["present"],
        "TEST": ["POssible"],
    })

nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]

In [3]:
medical.AssertionDLApproach().extractParamMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='AssertionDLApproach_4802d16346c4', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False, Param(parent='AssertionDLApproach_4802d16346c4', name='label', doc='Column with one label per document'): 'label', Param(parent='AssertionDLApproach_4802d16346c4', name='batchSize', doc='Size for each batch in the optimization process'): 64, Param(parent='AssertionDLApproach_4802d16346c4', name='epochs', doc='Number of epochs for the optimization process'): 5, Param(parent='AssertionDLApproach_4802d16346c4', name='learningRate', doc='Learning rate for the optimization process'): 0.0012, Param(parent='AssertionDLApproach_4802d16346c4', name='dropout', doc='Dropout at the output of each layer'): 0.05, Param(parent='AssertionDLApproach_4802d16346c4', name='maxSentLen', doc='Max length for an input sentence.'): 250, Param(parent='AssertionDLApproach_4802d16346c4', name='includeConfidence', doc='whether to include confidence scores in annotati

In [4]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models")     .setInputCols(["sentence", "ner_chunk", "embeddings"])     .setOutputCol("assertion")\n'

In [5]:
text = """
GENERAL: He is an elderly gentleman in no acute distress. He is sitting up in bed eating his breakfast. He is alert and oriented and answering questions appropriately.
HEENT: Sclerae showed mild arcus senilis in the right. Left was clear. Pupils are equally round and reactive to light. Extraocular movements are intact. Oropharynx is clear.
NECK: Supple. Trachea is midline. No jugular venous pressure distention is noted. No adenopathy in the cervical, supraclavicular, or axillary areas.
ABDOMEN: Soft and not tender. There may be some fullness in the left upper quadrant, although I do not appreciate a true spleen with inspiration.
EXTREMITIES: There is some edema, but no cyanosis and clubbing .
IMPRESSION: At this time is refractory anemia, which is transfusion dependent. He is on B12, iron, folic acid, and Procrit. There are no sign or symptom of blood loss and the previous esophagogastroduodenoscopy was negative. His creatinine was 1.
  My impression at this time is that he probably has an underlying myelodysplastic syndrome or bone marrow failure. His creatinine on this hospitalization was up slightly to 1.6 and this may contribute to his anemia.
  At this time, my recommendation for the patient is that he should undergo a bone marrow aspiration.
  I have discussed the procedure in detail which the patient. I have discussed the risks, benefits, and successes of that treatment and usefulness of the bone marrow and predicting his cause of refractory anemia and further therapeutic interventions, which might be beneficial to him.
  He is willing to proceed with the studies I have described to him. We will order an ultrasound of his abdomen because of the possible fullness of the spleen.
  As always, we greatly appreciate being able to participate in the care of your patient. We appreciate the consultation of the patient.
"""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
light_model = nlp.LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])
pd.set_option('display.max_columns', None)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                               chunks                   entities assertion  \
0                            distress                    Symptom    Absent   
1                       arcus senilis  Disease_Syndrome_Disorder      Past   
2  jugular venous pressure distention                    Symptom    Absent   
3                          adenopathy                    Symptom    Absent   
4                              tender                    Symptom    Absent   

  confidence  
0     0.9999  
1        1.0  
2        1.0  
3        1.0  
4        1.0

📌 Since notebook runs on EMR cluster, we can display visualization on local by saving the visualization html.

In [10]:
vis = nlp.viz.AssertionVisualizer()

#vis.display(light_result, 'ner_chunk', 'assertion')
vis.display(light_result, 'ner_chunk', 'assertion',save_path="/tmp/display_result.html")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<IPython.core.display.HTML object>

In [11]:
%%local
from IPython.display import display,HTML
with open("/tmp/display_result.html",'r') as display_result:
    display(HTML(display_result.read()))

In [12]:
nlp.nlu.to_pretty_df(model,text,output_level='chunk').columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['assertion', 'assertion_confidence', 'document', 'entities_ner_chunk',
       'entities_ner_chunk_class', 'entities_ner_chunk_confidence',
       'entities_ner_chunk_origin_chunk', 'entities_ner_chunk_origin_sentence',
       'sentence_pragmatic', 'word_embedding_embeddings'],
      dtype='object')

In [13]:
cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

df = nlp.nlu.to_pretty_df(model,text,output_level='chunk')[cols].reset_index(drop=True)
spark.createDataFrame(df).show(truncate=50)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+-------------------------+------------+--------------------+
|                entities_ner_chunk| entities_ner_chunk_class|   assertion|assertion_confidence|
+----------------------------------+-------------------------+------------+--------------------+
|                          distress|                  Symptom|      Absent|              0.9999|
|                     arcus senilis|Disease_Syndrome_Disorder|        Past|                 1.0|
|jugular venous pressure distention|                  Symptom|      Absent|                 1.0|
|                        adenopathy|                  Symptom|      Absent|                 1.0|
|                            tender|                  Symptom|      Absent|                 1.0|
|                          fullness|                  Symptom|    Possible|                 1.0|
|                             edema|                  Symptom|     Present|                 1.0|
|                          cya

In [ ]:
# Downloading sample datasets.
#! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples_10.csv

In [14]:
%%sh
wget -q -O /tmp/mt_samples_10.csv https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples_10.csv

In [15]:
mt_samples_df = spark.createDataFrame(pd.read_csv("/tmp/mt_samples_10.csv", sep=',', index_col=["index"]).reset_index())

mt_samples_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: long (nullable = true)
 |-- text: string (nullable = true)

In [16]:
mt_samples_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|index|                text|
+-----+--------------------+
|    0|Sample Type / Med...|
|    1|Sample Type / Med...|
|    2|Sample Type / Med...|
|    3|Sample Type / Med...|
|    4|Sample Type / Med...|
|    5|Sample Type / Med...|
|    6|Sample Type / Med...|
|    7|Sample Type / Med...|
|    8|Sample Type / Med...|
|    9|Sample Type / Med...|
+-----+--------------------+

In [17]:
result = model.transform(mt_samples_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|           assertion|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 24...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 76, ...|[{assertion, 68, ...|
|    1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 26...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 92, ...|[{assertion, 68, ...|
|    2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 14...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_

In [19]:
result.select('sentence.result').take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(result=['Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', '(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.', 'SECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', 'PROCEDURES', '1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.', '2. On August 20, 2007, thoracentesis.', '3. On August 31, 2007, Port-A-Cath placement.', 'HISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week.', 'She has had right-sided chest pain radiating to her back with fever starting yesterday.', 'She has a history of pericarditis and pericardectomy in May 2006 and developed cough with righ

In [20]:
import pyspark.sql.functions as F

result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.assertion.result,
                                     result.assertion.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['sentence']").alias("sent_id"),
              F.expr("cols['4']").alias("assertion"),
              F.expr("cols['5']['confidence']").alias("confidence") ).show(truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+-----+---+-------------------------+-------+------------+----------+
|chunk                    |begin|end|ner_label                |sent_id|assertion   |confidence|
+-------------------------+-----+---+-------------------------+-------+------------+----------+
|Discharge                |68   |76 |Admission_Discharge      |0      |Past        |0.991     |
|pleural effusion         |132  |147|Disease_Syndrome_Disorder|0      |Present     |1.0       |
|anemia                   |171  |176|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|ascites                  |179  |185|Disease_Syndrome_Disorder|0      |Hypothetical|0.9782    |
|esophageal reflux        |188  |204|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|deep venous thrombosis   |222  |243|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|Pleural effusion         |340  |355|Disease_Syndrome_Disorder|2      |Present     |1.0       |
|anemia                   |379  |384|Dis

### Pretrained `assertion_dl_radiology` model

In [21]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model for radiology
radiology_ner = medical.NerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
radiology_assertion = medical.AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    radiology_ner,
    ner_converter,
    radiology_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_radiology download started this may take some time.
[OK!]
assertion_dl_radiology download started this may take some time.
[OK!]

In [22]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
data = spark.createDataFrame([[text]]).toDF("text")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
result = radiologyAssertion_model.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
import pyspark.sql.functions as F

result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata,
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+-------+---------+
|chunk              |ner_label      |sent_id|assertion|
+-------------------+---------------+-------+---------+
|effusion           |ImagingFindings|0      |Negative |
|pneumothorax       |ImagingFindings|0      |Negative |
|displaced fractures|ImagingFindings|0      |Confirmed|
+-------------------+---------------+-------+---------+

## Writing a generic Assertion + NER function

In [26]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line
    sentenceDetector = nlp.SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP
    tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
    word_embeddings = nlp.WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    base_pipeline = nlp.Pipeline(stages=[
                        documentAssembler,
                        sentenceDetector,
                        tokenizer,
                        word_embeddings])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, ner_model_name = 'ner_clinical', assertion_model_name="assertion_dl"):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = medical.NerModel.pretrained(ner_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = medical.NerConverterInternal() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = medical.AssertionDLModel.pretrained(assertion_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")


    base_model = get_base_pipeline (embeddings)

    nlpPipeline = nlp.Pipeline(stages=[
        base_model,
        loaded_ner_model,
        ner_converter,
        clinical_assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", F.monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                                     result.ner_chunk.metadata,
                                                     result.assertion.result,
                                                     result.assertion.metadata)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label"),
                              F.expr("cols['2']").alias("assertion"),
                              F.expr("cols['3']['confidence']").alias("confidence"))\
                      .filter("ner_label!='O'")

    return result_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical_large'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show(30,truncate=50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_clinical_large download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------------------+---------+---------+----------+
|                       chunk|ner_label|assertion|confidence|
+----------------------------+---------+---------+----------+
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|            pleural effusion|  PROBLEM|  present|    0.9998|
|         atrial fibrillation|  PROBLEM|  present|       1.0|
|                      anemia|  PROBLEM|  present|    0.9999|
|                     ascites|  PROBLEM|  present|    0.9999|
|           esophageal reflux|  PROBLEM|  present|    0.9999|
|      deep venous thrombosis|  PROBLEM|  present|    0.8533|
|                Mesothelioma|  PROBLEM|  present|    0.9992|
|            Pleural eff

In [28]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_posology download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+------------+----------+
|           chunk|ner_label|   assertion|confidence|
+----------------+---------+------------+----------+
|        Coumadin|     DRUG|hypothetical|    0.8709|
|            1 mg| STRENGTH| conditional|    0.7772|
|           daily|FREQUENCY| conditional|    0.5086|
|      Amiodarone|     DRUG|hypothetical|    0.8589|
|          100 mg| STRENGTH|hypothetical|    0.6143|
|             p.o|    ROUTE|hypothetical|    0.7991|
|           daily|FREQUENCY|     present|    0.9074|
|        Coumadin|     DRUG|     present|    0.9999|
|         Lovenox|     DRUG|     present|    0.9994|
|           40 mg| STRENGTH|     present|    0.9982|
|  subcutaneously|    ROUTE|     present|    0.9302|
|    chemotherapy|     DRUG|    

In [29]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil."]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_posology_greedy download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+---------+----------+
|           chunk|ner_label|assertion|confidence|
+----------------+---------+---------+----------+
|capsule of Advil|     DRUG|   absent|    0.9855|
+----------------+---------+---------+----------+

In [30]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical'

entry_data = spark.createDataFrame([["The patient has no fever"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_clinical download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+-----+---------+---------+----------+
|chunk|ner_label|assertion|confidence|
+-----+---------+---------+----------+
|fever|  PROBLEM|   absent|     0.998|
+-----+---------+---------+----------+

In [31]:
def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]
  confidence=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

      chunks.append(n.result)
      entities.append(n.metadata['entity'])
      status.append(m.result)
      confidence.append(m.metadata['confidence'])

  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status,'confidence':confidence})

  return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
clinical_text = """
Patient with severe fever and sore throat.
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = nlp.LightPipeline(model)

# get_clinical_assertion_light (light_model, clinical_text)

cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

df = nlp.nlu.to_pretty_df(light_model,clinical_text, output_level='chunk')[cols]
spark.createDataFrame(df).show(truncate=50)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+-------------------------+------------+--------------------+
|         entities_ner_chunk| entities_ner_chunk_class|   assertion|assertion_confidence|
+---------------------------+-------------------------+------------+--------------------+
|                      fever|               VS_Finding|     Present|                 1.0|
|                sore throat|                  Symptom|     Present|                 1.0|
|               stomach pain|                  Symptom|      Absent|                 1.0|
|                       pain|                  Symptom|Hypothetical|                 1.0|
|            short of breath|                  Symptom|     Present|                 1.0|
|climbing a flight of stairs|                  Symptom|     Present|              0.9434|
|                  Alzheimer|Disease_Syndrome_Disorder|      Family|              0.8136|
+---------------------------+-------------------------+------------+--------------------+

# Oncological Assertion Models

Oncology Assertion Models

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
| 1 | [assertion_oncology_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_wip_en.html) | Medical_History, Family_History, Possible, Hypothetical_Or_Absent|
| 2 | [assertion_oncology_problem_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_problem_wip_en.html) |Present, Possible, Hypothetical, Absent, Family|
| 3 | [assertion_oncology_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_treatment_binary_wip_en.html) |Present, Planned, Past, Hypothetical, Absent|
| 3 | [assertion_oncology_treatment_wip]() |Present, Planned, Past, Hypothetical, Absent|
| 4 | [assertion_oncology_response_to_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_response_to_treatment_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 5 | [assertion_oncology_test_binary_wip](https://nlp.johnsnowlabs.com/2022/10/01/assertion_oncology_test_binary_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 6 | [assertion_oncology_smoking_status_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_smoking_status_wip_en.html) |Absent, Past, Present|
| 7 | [assertion_oncology_family_history_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_family_history_wip_en.html) |Family_History, Other|
| 8 | [assertion_oncology_demographic_binary_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_demographic_binary_wip_en.html) |Patient, Someone_Else|

In [34]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_oncology_wip'

assertion_model_name='assertion_oncology_wip'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name,assertion_model_name )

ner_df.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_oncology_wip download started this may take some time.
[OK!]
assertion_oncology_wip download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+------------------------+--------------------+------------+----------+
|chunk                   |ner_label           |assertion   |confidence|
+------------------------+--------------------+------------+----------+
|Mesothelioma            |Cancer_Dx           |Present     |0.9885    |
|Mesothelioma            |Cancer_Dx           |Hypothetical|0.981     |
|August 24, 2007         |Date                |Past        |0.9726    |
|decortication           |Cancer_Surgery      |Past        |0.994     |
|lung                    |Site_Lung           |Past        |0.9453    |
|pleural                 |Site_Other_Body_Part|Past        |0.9624    |
|biopsy                  |Pathology_Test      |Past        |0.9979    |
|transpleural fluoroscopy|Imaging_Test  

# Entity Type Constraints


You can effortlessly constrain assertions based on specific entity types using a convenient dictionary format: `{"entity": [assertion_label1, assertion_label2, .. assertion_labelN]}`. When an entity is not found in the dictionary, no constraints are applied, ensuring flexibility in your data processing. With the `setEntityAssertionCaseSensitive` parameter, you can control the case sensitivity for both entities and assertion labels. Unleash the full potential of your NLP model with these cutting-edge additions to the AssertionDLModel.

In [35]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = medical.NerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = medical.AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)\
    .setEntityAssertion({
        "PROBLEM": ["hypothetical", "absent"],
        "treAtment": ["present"],
        "TEST": ["POssible"],
    })

nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_clinical_large download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]

In [36]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
light_model = nlp.LightPipeline(model)

light_result = light_model.fullAnnotate(text)

chunks=[]
entities=[]
status=[]
confidence=[]

for assertion_row in light_result[0]["assertion"]:
  chunk_id = assertion_row.metadata["chunk"]
  for chunk_row in light_result[0]["ner_chunk"]:
    if chunk_id == chunk_row.metadata["chunk"]:
        chunks.append(chunk_row.result)
        entities.append(chunk_row.metadata['entity'])
        status.append(assertion_row.result)
        confidence.append(assertion_row.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})


df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                 chunks   entities     assertion confidence
0             metformin  TREATMENT       Present     0.5364
1             glipizide  TREATMENT       Present     0.9993
2         dapagliflozin  TREATMENT       Present        1.0
3                   HTG    PROBLEM  Hypothetical        1.0
4  Physical examination       TEST      Possible      0.943
5            tenderness    PROBLEM        Absent        1.0
6              guarding    PROBLEM        Absent        1.0
7              rigidity    PROBLEM  Hypothetical     0.9999

# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like present or conditional; and do not want absent conditions get out of your pipeline.

In [45]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

clinical_assertion = medical.AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_filterer = medical.AssertionFilterer()\
    .setInputCols("sentence","ner_chunk","assertion")\
    .setOutputCol("assertion_filtered")\
    .setCaseSensitive(False)\
    .setWhiteList(["PREsent"])

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      clinical_assertion,
      assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]

In [46]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = nlp.LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [47]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('a headache', 'Absent')]

In [48]:
assertion_filterer.getWhiteList()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['PREsent']

In [49]:
chunks=[]
entities=[]
status=[]
confidence=[]

light_result = light_model.fullAnnotate(text)[0]

for m in light_result['assertion_filtered']:

    chunks.append(m.result)
    entities.append(m.metadata['entity'])
    status.append(m.metadata['assertion'])
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [chunks, entities, assertion, confidence]
Index: []

As you see, there is no "pain" chunk since it has "absent" assertion label.

# AssertionChunkConverter

In some cases, there may be issues while creating the chunk column by using token indices and losing some data while training and testing the assertion status model if there are issues in these token indices. So we developed a new `AssertionChunkConverter` annotator that takes **begin and end indices of the chunks** as input and creates an extended chunk column with metadata that can be used for assertion status detection model training.

*NOTE*: Chunk begin and end indices in the assertion status model training dataframe can be populated using the new version of ALAB module.

In [50]:
data = spark.createDataFrame([["An angiography showed bleeding in two vessels off of the Minnie supplying the sigmoid that were succesfully embolized.", "Minnie", 57, 63],
     ["After discussing this with his PCP, Leon was clear that the patient had had recurrent DVTs and ultimately a PE and his PCP felt strongly that he required long-term anticoagulation ", "PCP", 31, 34]])\
     .toDF("text", "target", "char_begin", "char_end")

data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+----------+--------+
|                text|target|char_begin|char_end|
+--------------------+------+----------+--------+
|An angiography sh...|Minnie|        57|      63|
|After discussing ...|   PCP|        31|      34|
+--------------------+------+----------+--------+

In [51]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("tokens")

converter = medical.AssertionChunkConverter() \
    .setInputCols("tokens")\
    .setChunkTextCol("target")\
    .setChunkBeginCol("char_begin")\
    .setChunkEndCol("char_end")\
    .setOutputTokenBeginCol("token_begin")\
    .setOutputTokenEndCol("token_end")\
    .setOutputCol("chunk")

pipeline = nlp.Pipeline().setStages([document_assembler,sentenceDetector, tokenizer, converter])

results = pipeline.fit(data).transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
results\
    .selectExpr(
        "target",
        "char_begin",
        "char_end",
        "token_begin",
        "token_end",
        "tokens[token_begin].result",
        "tokens[token_end].result",
        "target",
        "chunk")\
    .show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|target|char_begin|char_end|token_begin|token_end|tokens[token_begin].result|tokens[token_end].result|target|chunk                                         |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|Minnie|57        |63      |10         |10       |Minnie                    |Minnie                  |Minnie|[{chunk, 57, 62, Minnie, {sentence -> 0}, []}]|
|PCP   |31        |34      |5          |5        |PCP                       |PCP                     |PCP   |[{chunk, 31, 33, PCP, {sentence -> 0}, []}]   |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+

# Train a custom Assertion Model

In [53]:
#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
%%sh
wget -q -O /tmp/i2b2_assertion_sample_short.csv https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv
hdfs dfs -copyFromLocal /tmp/i2b2_assertion_sample_short.csv /user/hadoop/i2b2_assertion_sample_short.csv

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [55]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/user/hadoop/i2b2_assertion_sample_short.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
assertion_df.show(3, truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------+-------------------+-------+-----+---+
|                                             text|             target|  label|start|end|
+-------------------------------------------------+-------------------+-------+-----+---+
|She has no history of liver disease , hepatitis .|      liver disease| absent|    5|  6|
|                         1. Undesired fertility .|undesired fertility|present|    1|  2|
|                            3) STATUS POST FALL .|               fall|present|    3|  3|
+-------------------------------------------------+-------------------+-------+-----+---+
only showing top 3 rows

In [57]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 721
Test Dataset Count: 170

In [58]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
|label  |count|
+-------+-----+
|present|546  |
|absent |175  |
+-------+-----+

In [59]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = nlp.Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = nlp.Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]

We will transform our test data with a pipeline consisting of same steps with the pipeline which contains AssertionDLApproach.
By doing this, we enable that test data will have same columns with training data in AssertionDLApproach. <br/>
The goal of this implementation is enabling the usage of `setTestDataset()` parameter in AssertionDLApproach.

In [60]:
clinical_assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings])

assertion_test_data = clinical_assertion_pipeline.fit(test_data).transform(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
assertion_test_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['text', 'target', 'label', 'start', 'end', 'document', 'chunk', 'token', 'embeddings']

We save the test data in parquet format to use in `AssertionDLApproach()`.

In [62]:
assertion_test_data.write.parquet('i2b2_assertion_sample_test_data.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Graph setup and training

In [63]:
#!pip install -q tensorflow==2.11.0
#!pip install -q tensorflow-addons

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline.

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [65]:
graph_folder= "tfgraphs"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
assertion_graph_builder = medical.TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(250)\
    .setHiddenUnitsNumber(25)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
'''
# ready to use tf_graph

!mkdir training_logs
!mkdir assertion_tf_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\n# ready to use tf_graph\n\n!mkdir training_logs\n!mkdir assertion_tf_graph\n\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph\n'

In [68]:
'''
# create custom graph

medical.tf_graph.print_model_params("assertion_dl")

feat_size = 200
n_classes = 6

medical.tf_graph.build("assertion_dl",
                        build_params={"n_classes": n_classes},
                        model_location= "./tf_graphs",
                        model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\n# create custom graph\n\nmedical.tf_graph.print_model_params("assertion_dl")\n\nfeat_size = 200\nn_classes = 6\n\nmedical.tf_graph.build("assertion_dl",\n                        build_params={"n_classes": n_classes},\n                        model_location= "./tf_graphs",\n                        model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))\n'

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [69]:
scope_window = [10,10]

assertionStatus = medical.AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(64)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(20)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setIncludeConfidence(True)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setScopeWindow(scope_window)\
   # .setTestDataset(path="/content/i2b2_assertion_sample_test_data.parquet")


'''
If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
by a pipeline consisting of document, chunk, embeddings stages.
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nIf .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed\nby a pipeline consisting of document, chunk, embeddings stages.\n'

In [70]:
'''
assertionStatus = medical.AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nassertionStatus = medical.AssertionLogRegApproach()    .setLabelCol("label")    .setInputCols("document", "chunk", "embeddings")    .setOutputCol("assertion")    .setMaxIter(100) # default: 26\n'

In [71]:
clinical_assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertion_graph_builder,
    assertionStatus])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: tfgraphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 2, 'feat_size': 200, 'max_seq_len': 250, 'n_hidden': 25}
assertion_dl graph exported to tfgraphs/assertion_graph.pb

## Checking the results

Checking the results saved in the log file

In [73]:
%%sh
hdfs dfs -ls /user/livy/training_logs/AssertionDLApproach_*.log |sort -nr

-rw-r--r--   1 livy livy      12078 2023-09-13 11:51 /user/livy/training_logs/AssertionDLApproach_ad6ce2d83fbc.log


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [74]:
%%sh
hdfs dfs -cat  /user/livy/training_logs/AssertionDLApproach_ad6ce2d83fbc.log

Name of the selected graph: tfgraphs/assertion_graph.pb
Training started, trainExamples: 721


Epoch: 0 started, learning rate: 0.001, dataset size: 577
Done, 5.107235819 total training loss: 5.029212, avg training loss: 0.5029212, batches: 10
Quality on validation dataset (20.0%), validation examples = 144
time to finish evaluation: 1.09s
Total validation loss: 1.8882	Avg validation loss: 0.6294
label	 tp	 fp	 fn	 prec	 rec	 f1
present	 101	 43	 0	 0.7013889	 1.0	 0.82448983
absent	 0	 0	 43	 0.0	 0.0	 0.0
tp: 101 fp: 43 fn: 43 labels: 2
Macro-average	 prec: 0.35069445, rec: 0.5, f1: 0.41224492
Micro-average	 prec: 0.7013889, rec: 0.7013889, f1: 0.7013889


Epoch: 1 started, learning rate: 9.5E-4, dataset size: 577
Done, 3.76468277 total training loss: 6.1905203, avg training loss: 0.61905205, batches: 10
Quality on validation dataset (20.0%), validation examples = 144
time to finish evaluation: 0.73s
Total validation loss: 1.8025	Avg validation loss: 0.6008
label	 tp	 fp	 fn	 prec	 r

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [75]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+
|  label|   result|
+-------+---------+
|present|[present]|
| absent|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
+-------+---------+
only showing top 20 rows

In [76]:
preds_df = preds.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
preds_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       label   result
0    present  present
1     absent  present
2    present  present
3    present  present
4    present  present
..       ...      ...
165  present  present
166   absent   absent
167   absent   absent
168   absent   absent
169  present  present

[170 rows x 2 columns]

In [78]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['label'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

      absent       0.90      0.72      0.80        53
     present       0.88      0.97      0.92       117

    accuracy                           0.89       170
   macro avg       0.89      0.84      0.86       170
weighted avg       0.89      0.89      0.88       170

In [79]:
# save model
assertion_model.stages[-1].write().overwrite().save('assertion_custom_model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load saved model

In [80]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]

In [81]:
clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

clinical_assertion = medical.AssertionDLModel.load("assertion_custom_model") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ner_clinical download started this may take some time.
[OK!]

In [82]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia noted. She denies pain'

light_model = nlp.LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

print(text)

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia noted. She denies pain
       chunks entities assertion confidence
0  a headache  PROBLEM   present     0.9998
1   a head CT     TEST   present     0.9998
2     anxious  PROBLEM   present     0.5182
3    alopecia  PROBLEM    absent     0.6481
4        pain  PROBLEM   present     0.5916